In [ ]:
!pip install python-dotenv
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "at2507"
model_id = f"{username}/gemma-2b-instruct-ft-python-code-instructions"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [ ]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [ ]:
# query = """\n\n Please answer with one of the option in the bracket. Write reasoning in between <analysis></analysis>.
#            Write answer in between <answer></answer>. here are the inputs Q:An 8-year-old boy is brought to the pediatrician
#            by his mother with nausea, vomiting, and decreased frequency of urination. He has acute lymphoblastic leukemia
#            for which he received the 1st dose of chemotherapy 5 days ago. His leukocyte count was 60,000/mm3 before starting
#            chemotherapy. The vital signs include: pulse 110/min, temperature 37.0°C (98.6°F), and blood pressure 100/70 mm Hg.
#            The physical examination shows bilateral pedal edema. Which of the following serum studies and urinalysis findings
#            will be helpful in confirming the diagnosis of this condition? ? \n{'A': 'Hyperkalemia, hyperphosphatemia, hypocalcemia,
#            and extremely elevated creatine kinase (MM)', 'B': 'Hyperkalemia, hyperphosphatemia, hypocalcemia, hyperuricemia, urine
#            supernatant pink, and positive for heme', 'C': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, lactic acidosis, an
#             urate crystals in the urine', 'D': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, and urinary monoclonal spike',
#             'E': 'Hyperuricemia, hyperkalemia, hyperphosphatemia, lactic acidosis, and oxalate crystals'}"""

# query = """Create a function to calculate the sum of a sequence of integers.
# """

query = " Create a for loop in Python that prints the output of a multiplication table for numbers from 1 to 10. here are the inputs -"

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
   Create a for loop in Python that prints the output of a multiplication table for numbers from 1 to 10. here are the inputs -
  
model


  for x in range (1,11): 
 print(x*x) 
 
  

1
2
3
4
5
6
7
8
9
10

 
   
 
  The output is: 
   1 
   4 
   9 
    16 
    25 
    36 
    49 
    64 
    81 
    100 
 Y
 
  
 
  The total of the iterations is 10. 
   

The for loop is 10 because from 1 to 11 there are ten numbers. 
   
The variable 'x' is bound to the value on the right of the for statement, we use range(1,11) to create a variable that will loop from 1 to 10. 
   
The variable 'x' will be incremented with each iteration. 
   
The print statement prints out the value of x in each iteration. 

The result is the multiplication table for numbers from 1 to 10. 
 
  
 

Here is an alternative solution without using a loop: 

  
def table(num_):
 table = [] 
 for i in range

In [ ]:
# print(f"{result.split('Analysis:')[-1]}")
print(f"{result.split('model')[-1]}")




  for x in range (1,11): 
 print(x*x) 
 
  

1
2
3
4
5
6
7
8
9
10

 
   
 
  The output is: 
   1 
   4 
   9 
    16 
    25 
    36 
    49 
    64 
    81 
    100 
 Y
 
  
 
  The total of the iterations is 10. 
   

The for loop is 10 because from 1 to 11 there are ten numbers. 
   
The variable 'x' is bound to the value on the right of the for statement, we use range(1,11) to create a variable that will loop from 1 to 10. 
   
The variable 'x' will be incremented with each iteration. 
   
The print statement prints out the value of x in each iteration. 

The result is the multiplication table for numbers from 1 to 10. 
 
  
 

Here is an alternative solution without using a loop: 

  
def table(num_):
 table = [] 
 for i in range(1,num_+1):
 row = []
 for j in range(1,num+1):
  row.append((i * j))
  table.append(row)
 return table 
 
result = table(10) 
 print(result) 
   
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
  
 
 <unused99>
 

The total complexity of this solution is 0(14) with 